# Create RO-Crate from NIRD dataset

In [1]:
pip install rocrate

  Using cached rocrate-0.13.0-py3-none-any.whl.metadata (18 kB)
  Using cached arcp-0.2.1-py2.py3-none-any.whl.metadata (7.6 kB)
Using cached rocrate-0.13.0-py3-none-any.whl (332 kB)
Using cached arcp-0.2.1-py2.py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
from rocrate.rocrate import ROCrate
from rocrate.model.person import Person
import pandas as pd
from datetime import datetime
import geopandas
import shapely
import xarray as xr
import numpy as np

## Open RiOMar data to get metadata

In [3]:
url = "https://data-fair2adapt.ifremer.fr/riomar/small.zarr"

In [4]:
ds = xr.open_zarr(url)
ds

<xarray.Dataset> Size: 492MB
Dimensions:       (y_rho: 838, x_rho: 727, s_rho: 40, time_counter: 5)
Coordinates:
    nav_lat_rho   (y_rho, x_rho) float32 2MB dask.array<chunksize=(838, 727), meta=np.ndarray>
    nav_lon_rho   (y_rho, x_rho) float32 2MB dask.array<chunksize=(838, 727), meta=np.ndarray>
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * time_counter  (time_counter) datetime64[ns] 40B 2004-01-01T00:58:30 ... 2...
    time_instant  (time_counter) datetime64[ns] 40B dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: y_rho, x_rho
Data variables:
    temp          (time_counter, s_rho, y_rho, x_rho) float32 487MB dask.array<chunksize=(1, 40, 838, 727), meta=np.ndarray>
Attributes: (12/45)
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...
    Conventions:    CF-1.6
    Cs_r:           have a look at variable Cs_r in this file
    Cs_w:           have a look at variable Cs_w in this file
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    Tcline:         15.0
    ...             ...
    title:          GAMAR_GLORYS
    tnu4_expl:      biharmonic mixing coefficient for tracers
    units:          meter4 second-1
    uuid:           06f6b784-fcc0-4422-aceb-17da2a5aa9fa
    v_sponge:       0.0
    x_sponge:       0.0

## Get metadata from RiOMAR

### Get bounding box in WKT
- Latitudes with values of -1 are NaN

In [5]:
minlat = ds.nav_lat_rho.where(ds.nav_lat_rho > -1, np.nan).min().values
maxlat = ds.nav_lat_rho.max().values
minlon = ds.nav_lon_rho.min().values
maxlon = ds.nav_lon_rho.max().values
print(minlat, maxlat, minlon, maxlon)

43.285 50.86747 -8.0 1.68


In [6]:
geometry_wkt = shapely.geometry.box(minlon, minlat, maxlon, maxlat).wkt
geometry_wkt

'POLYGON ((1.6799999475479126 43.28499984741211, 1.6799999475479126 50.867469787597656, -8 50.867469787597656, -8 43.28499984741211, 1.6799999475479126 43.28499984741211))'

- time range 

In [7]:
ts = pd.to_datetime(str(ds.time_counter.min().values)) 
te = pd.to_datetime(str(ds.time_counter.max().values)) 
date_start = ts.strftime('%Y.%m.%d')
date_end = te.strftime('%Y.%m.%d')
date_start, date_end

('2004.01.01', '2004.01.01')

### Get information for the provenance

In [8]:
prov = {
      "@id": "https://doi.org/10.5281/zenodo.13898339",
      "@type": "SoftwareApplication",
      "url": "https://www.croco-ocean.org",
      "name": "CROCO, Coastal and Regional Ocean COmmunity",
      "version": "CROCO GAMA model v2.0.1 https://doi.org/10.5281/zenodo.13898339"
}

## Get Institutions

In [10]:
filename = "./instituion-v3.json"
with open(filename, 'r') as file:
    data = file.read()
    institutions = json.loads(data)
institutions = pd.DataFrame.from_dict(institutions)
institutions.head()

,id,name,realm,support_url,support_email,type,ror
0,54,"{'nb': 'Politihøgskolen', 'nn': 'Politihøgskul...",phs.no,{},it@phs.no,"[home_organization, higher_education, service_...",https://ror.org/05486d596
1,55,"{'nb': 'Universitetet i Agder', 'nn': 'Univers...",uia.no,{'nb': 'https://www.uia.no/student/uia-hjelp/b...,hjelp@uia.no,"[home_organization, higher_education, service_...",https://ror.org/03x297z98
2,58,"{'nb': 'NIFU', 'nn': 'NIFU', 'en': 'NIFU'}",nifu.no,{},bibliotek@nifu.no,"[home_organization, higher_education]",https://ror.org/032jh6m87
3,61,"{'nb': 'Universitetssenteret på Svalbard', 'en...",unis.no,{},support@unis.no,"[home_organization, higher_education]",https://ror.org/03cyjf656
4,70,"{'nb': 'Høgskolen i Østfold', 'nn': 'Høgskulen...",hiof.no,{'nb': 'https://www.hiof.no/tjenester/it/'},itvakt@hiof.no,"[home_organization, higher_education, service_...",https://ror.org/04gf7fp41


### Split name in several columns

In [11]:
rename_cols = {}
for name in institutions["name"].apply(pd.Series).columns:
    rename_cols[name] = "name-" + name
rename_cols

{'nb': 'name-nb', 'nn': 'name-nn', 'en': 'name-en', 'se': 'name-se'}

In [12]:
institutions = institutions.join( institutions["name"].apply(pd.Series))
institutions = institutions.rename(columns=rename_cols)
institutions.head()

,id,name,realm,support_url,support_email,type,ror,name-nb,name-nn,name-en,name-se
0,54,"{'nb': 'Politihøgskolen', 'nn': 'Politihøgskul...",phs.no,{},it@phs.no,"[home_organization, higher_education, service_...",https://ror.org/05486d596,Politihøgskolen,Politihøgskulen,The Norwegian Police University College,NaN
1,55,"{'nb': 'Universitetet i Agder', 'nn': 'Univers...",uia.no,{'nb': 'https://www.uia.no/student/uia-hjelp/b...,hjelp@uia.no,"[home_organization, higher_education, service_...",https://ror.org/03x297z98,Universitetet i Agder,Universitetet i Agder,University of Agder,Agder Universitehta
2,58,"{'nb': 'NIFU', 'nn': 'NIFU', 'en': 'NIFU'}",nifu.no,{},bibliotek@nifu.no,"[home_organization, higher_education]",https://ror.org/032jh6m87,NIFU,NIFU,NIFU,NaN
3,61,"{'nb': 'Universitetssenteret på Svalbard', 'en...",unis.no,{},support@unis.no,"[home_organization, higher_education]",https://ror.org/03cyjf656,Universitetssenteret på Svalbard,NaN,The University Center in Svalbard,NaN
4,70,"{'nb': 'Høgskolen i Østfold', 'nn': 'Høgskulen...",hiof.no,{'nb': 'https://www.hiof.no/tjenester/it/'},itvakt@hiof.no,"[home_organization, higher_education, service_...",https://ror.org/04gf7fp41,Høgskolen i Østfold,Høgskulen i Østfold,Østfold University College,Østfold Allaskuvla


## Access the NIRD Archive via its API

In [13]:
try:
    with open("./nird.json") as config:
        token = json.load(config)["token"]
except FileNotFoundError:
    print("config.json not found!")

In [15]:
url = "https://admin.ckan-archive-test.sigma2.no/api/3/action/package_search"
response = requests.get(url, headers={ "Authorization": token})

list_records = response.json()["result"]["results"]
list_records

[{'access_rights': 'non-public',
  'author': None,
  'author_email': None,
  'creator_user_id': '3f8539aa-a856-4bfb-9bdd-254b46d03fd9',
  'cron': {'state': 'succeeded',
   'message': '',
   'submitted_date': '2025-01-17T18:30:23',
   'completed_date': '2025-01-17T18:36:00'},
  'dataset_status': 'complete',
  'doi': '10.82969/2025.q7cgkhry',
  'id': '740fd40d-c809-4dde-92f0-21f1fc9f16fe',
  'import_done': True,
  'import_source': 'project_area',
  'isopen': True,
  'language': 'en',
  'license_id': 'CC-BY-4.0',
  'license_title': 'Creative Commons Attribution 4.0',
  'license_url': 'https://creativecommons.org/licenses/by/4.0/',
  'maintainer': None,
  'maintainer_email': None,
  'metadata_created': '2025-01-17T15:39:44.149643',
  'metadata_modified': '2025-01-17T17:36:00.798491',
  'name': 'cesm2_bc_restartfiles',
  'notes': 'Restartfiles to run CESM2 from 1950 for 20 ensemble members',
  'num_resources': 0,
  'num_tags': 0,
  'organization': None,
  'owner_org': None,
  'private': Fal

## DOI to find in the NIRD archive

In [57]:
doi = "10.82969/2025.hkfi3bn9.v2"

In [59]:
for rec in (x for x in list_records if x["doi"] == doi):
    pass

In [60]:
print(rec)

{'access_rights': 'public', 'author': None, 'author_email': None, 'conforms_to': 'https://w3id.org/ro/crate/1.1', 'creator_user_id': '566de054-6efa-40ee-ba2a-373ef6e1657f', 'cron': {'state': 'succeeded', 'message': '', 'submitted_date': '2025-01-04T12:51:02', 'completed_date': '2025-01-04T12:51:05'}, 'dataset_status': 'in_work', 'doi': '10.82969/2025.u7avlttz', 'id': 'fb71af3f-b9fd-437d-9d4a-c9d9450d6727', 'import_done': True, 'import_source': 'local', 'isopen': True, 'language': 'en', 'license_id': 'CC-BY-4.0', 'license_title': 'Creative Commons Attribution 4.0', 'license_url': 'https://creativecommons.org/licenses/by/4.0/', 'maintainer': None, 'maintainer_email': None, 'metadata_created': '2025-01-04T11:49:02.714258', 'metadata_modified': '2025-01-04T11:51:05.770223', 'name': 'test-for-ro-crate', 'notes': 'This dataset is meant to test the creation of RO-Crate. It contains one netcdf file.', 'num_resources': 2, 'num_tags': 2, 'organization': None, 'owner_org': None, 'private': False,

In [52]:
rec

{'access_rights': 'public',
 'author': None,
 'author_email': None,
 'conforms_to': 'https://w3id.org/ro/crate/1.1',
 'creator_user_id': '566de054-6efa-40ee-ba2a-373ef6e1657f',
 'cron': {'state': 'succeeded',
  'message': '',
  'submitted_date': '2025-01-04T12:51:02',
  'completed_date': '2025-01-04T12:51:05'},
 'dataset_status': 'in_work',
 'doi': '10.82969/2025.u7avlttz',
 'id': 'fb71af3f-b9fd-437d-9d4a-c9d9450d6727',
 'import_done': True,
 'import_source': 'local',
 'isopen': True,
 'language': 'en',
 'license_id': 'CC-BY-4.0',
 'license_title': 'Creative Commons Attribution 4.0',
 'license_url': 'https://creativecommons.org/licenses/by/4.0/',
 'maintainer': None,
 'maintainer_email': None,
 'metadata_created': '2025-01-04T11:49:02.714258',
 'metadata_modified': '2025-01-04T11:51:05.770223',
 'name': 'test-for-ro-crate',
 'notes': 'This dataset is meant to test the creation of RO-Crate. It contains one netcdf file.',
 'num_resources': 2,
 'num_tags': 2,
 'organization': None,
 'owne

## Create a new RO-Crate

In [23]:
crate = ROCrate()

## Add the license for the RO-Crate

In [39]:
crate.update_jsonld(
{
    "@id": "./",
    "license": { "@id":  rec["license_url"]},
})
license = {
                "@id": rec["license_url"],
                "@type": "CreativeWork",
                "name": rec["license_id"],
                "description": rec["license_title"],
                }
crate.add_jsonld(license)

ValueError: entity https://creativecommons.org/licenses/by/4.0/ already exists in the RO-Crate

## Add creators and their Organizations

We also need the ROR to create an affiliation for each organization found

In [25]:
def get_ror(institutions, org):
    ror = {}
    for name in rename_cols.values():
        if not institutions.loc[institutions[name].isin([org])].empty:
            ror_url = institutions.loc[institutions[name].isin([org])]["ror"].values[0]
            ror = {
                "@id": ror_url,
                "@type": "Organization",
                "name": org,
                "url": ror_url
                }
    return ror

In [26]:
list_authors = []
list_orcids = []
for creator in rec["creators"]:
    creator['givenName'] = creator.pop('first_name')
    creator['familyName'] = creator.pop('last_name') 
    list_authors.append(creator['givenName'] + " " +  creator['familyName'])
    org_name = creator.pop("organisation")
    ror = get_ror(institutions, org_name)
    creator["affiliation"] = {"@id": ror["url"]}
    orcid = creator.pop("orcid")
    print(creator)
    crate.add(Person(crate, "https://orcid.org/" + orcid, properties=creator))
    list_orcids.append({ "@id":  "https://orcid.org/" + orcid })
    crate.add_jsonld(ror)

# Add the list of authors
crate.update_jsonld({
    "@id": "./",
    "author": list_orcids,
})

rec.pop("creators")

{'email': 'annef@simula.no', 'givenName': 'Anne', 'familyName': 'Fouilloux', 'affiliation': {'@id': 'https://ror.org/00vn06n10'}}


[{'email': 'annef@simula.no',
  'givenName': 'Anne',
  'familyName': 'Fouilloux',
  'affiliation': {'@id': 'https://ror.org/00vn06n10'}}]

## Add data

In [27]:
for toc in (x for x in rec["resources"] if x["name"] == "table_of_contents"):
    pass
toc["url"]

'https://admin.ckan-archive-test.sigma2.no/dataset/3a818316-f498-43eb-958e-44b13dbed2b0/download/table_of_contents_10.82969_2025.hkfi3bn9.csv'

In [28]:
for resource in rec["resources"]:
    print(resource["name"])

table_of_contents
summary_contents


In [29]:
df = pd.read_csv(toc["url"], sep="|")
df = df.drop([0, 0])
df.head()

,filename,format,size,modified_date,fixity,http_url,s3_url
1,/archive_public/3a818316-f498-43eb-958e-44b13d...,application/x-hdf; charset=binary,2252270.0,2025-01-16T23:21:32.160679366 +0100,20c0a761cb92ab8884f1e5c2728f8727,https://admin.ckan-archive-test.sigma2.no/dat...,/dataset/3a818316-f498-43eb-958e-44b13dbed2b0...


In [30]:
df.columns

Index(['filename ', ' format ', ' size ', ' modified_date ', ' fixity ',
       ' http_url ', ' s3_url'],
      dtype='object')

In [35]:
rec

{'access_rights': 'public',
 'author': None,
 'author_email': None,
 'creator_user_id': '566de054-6efa-40ee-ba2a-373ef6e1657f',
 'cron': {'state': 'succeeded',
  'message': '',
  'submitted_date': '2025-01-16T23:48:02',
  'completed_date': '2025-01-16T23:48:04'},
 'dataset_status': 'in_work',
 'doi': '10.82969/2025.hkfi3bn9',
 'id': '3a818316-f498-43eb-958e-44b13dbed2b0',
 'import_done': True,
 'import_source': 'local',
 'isopen': True,
 'language': 'en',
 'license_id': 'CC-BY-4.0',
 'license_title': 'Creative Commons Attribution 4.0',
 'license_url': 'https://creativecommons.org/licenses/by/4.0/',
 'maintainer': None,
 'maintainer_email': None,
 'metadata_created': '2025-01-16T22:21:16.875948',
 'metadata_modified': '2025-01-16T22:48:04.965992',
 'name': 'riomar-test-sample',
 'notes': 'data from RiOMAR (River Dominated Ocean Margin). This dataset is not in its original resolution but has been thinned (thin=20).',
 'num_resources': 2,
 'num_tags': 2,
 'organization': None,
 'owner_org

### Prepare Temporal coverage if available

In [31]:
if "temporal" in rec.keys():
    temporal_coverage = rec["temporal"]["start"] + "/" + rec["temporal"]["end"] 
else:
    temporal_coverage = None

### Prepare Spatial coverage if available

In [32]:
def get_geoshape(spatial):
    if spatial["spatial_type"] == "wkt":
        geo = shapely.wkt.loads(spatial["value"])
        if hasattr(geo, 'geoms'):
            # take the first one
             geo = geo.geoms[0]
        geo = geo.wkt.replace("POLYGON", "").replace("(","").replace(")","").strip()   
        geolocation = { "@type": "GeoShape", "@id": geo, "polygon": geo}
        return geolocation
    else:
        # Not implemented yet
        return None

if "spatial" in rec:
    geolocation = get_geoshape(rec["spatial"])

In [34]:
rec["spatial"]

KeyError: 'spatial'

### Go through each data and add it in the RO-Crate 

In [ ]:
# "https://admin.ckan-archive-test.sigma2.no" + 
for row in df.itertuples(index=False, name=None):
    resource2add = {df.columns[i].strip() : row[i] for i, _ in enumerate(row)}
    url = resource2add.pop("http_url").strip()
    resource2add["name"] = resource2add["filename"].strip()
    resource2add["s3_url"] = "s3:/" + resource2add["s3_url"].strip()
    resource2add["sdDatePublished"] = rec["cron"]["completed_date"]     # Is it correct?
    resource2add["dateCreated"] = rec["metadata_created"]
    resource2add["dateModified"] = rec["metadata_modified"]
    resource2add["contentSize"] = resource2add.pop("size")
    resource2add["encodingFormat"] = resource2add.pop("format")
    if geolocation is not None:
        resource2add["location"] = geolocation
    if temporal_coverage is not None:
        resource2add["temporalCoverage"] = temporal_coverage
    
    ## properties we remove because we do not know where to fit them
    resource2add.pop("filename")   # was put in "name"
    resource2add.pop("fixity") # Checksum?
    resource2add.pop("s3_url")  # may be ut in different distribution but only valid for datasets and not files
    print("properties = ", resource2add)
    resource = crate.add_file(url, fetch_remote = False, properties=resource2add)

## Add metadata to RO

### Add the title and description

In [ ]:
title = rec.pop("title")
description = rec.pop("notes")
crate.update_jsonld({
    "@id": "./",
    "description": description,
    "title": title,
    "name": title,
})

### Add the publisher and creator

In [ ]:
publisher_name = rec.pop("publisher")  ## WEIRD! 
publisher_name = "Sigma2 AS"
publisher_url = "https://www.wikidata.org/wiki/Q12008197"
publisher = {
                "@id": publisher_url,
                "@type": "Organization",
                "name": publisher_name,
                "url": publisher_url
                }
crate.add_jsonld(publisher)
crate.update_jsonld(
{
    "@id": "./",
    "publisher": { "@id": publisher_url },
})

### Add the creator of the RO-Crate

In [ ]:
crate.update_jsonld(
{
    "@id": "ro-crate-metadata.json",
    "creator": { "@id": publisher_url },
})

### Add Publication date

In [ ]:
date_published =  datetime.strptime(rec.pop("release_date"), '%Y-%m-%dT%H:%M:%S.%f')

crate.update_jsonld({
    "@id": "./",
    "datePublished":  date_published.strftime("%Y-%m-%d") ,
})

### Add citation

In [ ]:
doi = "https://doi.org/" + rec.pop("doi")
cite_as = " and ".join(list_authors) + ", " + title + ", " + publisher_name + ", " + date_published.strftime("%Y") + ". " +  doi + "."

crate.update_jsonld({
    "@id": "./",
    "identifier": doi,
    "url": doi,
    "cite-as":  cite_as ,
})


### Add studySubject, keywords, etc.

In [ ]:
def find_subject(subject, groups):
    for group in groups:
        if subject == group["id"]:
            return group
    return None

In [ ]:
study_subjects = []
for subject in rec["subject"]:
    info_on_subject = find_subject(subject, rec["groups"])     
    study_subjects.append({
         "@id": "http://inspire.ec.europa.eu/metadata-codelist/TopicCategory/" + info_on_subject["name"]
    })

In [ ]:
keywords = []
for tag in rec["tags"]:
    keywords.append(tag["display_name"])
    
keywords = ", ".join(keywords)

crate.update_jsonld({
    "@id": "./",
    "about": study_subjects,
    "keywords":  keywords,
})

### Add version

In [ ]:
crate.update_jsonld({
    "@id": "./",
    "version": rec["version"],
})

### Add Language

In [ ]:
#crate.update_jsonld({
#    "@id": ,
#    "@type": "Language",
#})

In [ ]:
rec.keys()

In [ ]:
rec["theme"]

## Write to disk

In [ ]:
crate.write("ro-crate")

In [ ]:
from rocrateValidator import validate as validate

In [ ]:
v = validate.validate("ro-crate")
v.validator()